In [9]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console


In [7]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""당신은 명확성과 단순함에 초점을 맞춘 전문 편집자입니다.
            당신의 일은 모호함과 중복을 제거하고, 모든 문장을 간결하고 명확하게 만드는 것입니다.
            설득이나 톤은 걱정하지 말고 — 한국어 메시지를 읽기 쉽고 이해하기 쉽게 만드는 데 집중하세요.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""당신은 감정적 톤과 전문성에 집중하는 커뮤니케이션 코치입니다.
            이메일이 따뜻하고, 자신감 있으며, 인간적으로 들리도록 하되 — 대상에 적합한 전문성을 유지하세요.
            감정적 공명을 높이고, 표현을 다듬고, 딱딱하거나 차갑거나 지나치게 캐주얼하게 들릴 수 있는 단어를 조정하세요.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""당신은 마케팅, 행동심리학, 카피라이팅에 능통한 설득 전문가입니다.
            이메일의 설득력을 강화하세요: 행동 유도(Call to Action)를 개선하고, 논증 구조를 다듬으며, 이점을 강조하세요. 약하거나 수동적인 표현은 제거하세요.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""당신은 고급 이메일 작성 전문가입니다. 당신의 역할은 이전 에이전트의 모든
            응답과 수정안을 읽고, 최고의 아이디어를 하나의 통합되고 정제된 이메일 초안으로 **합성**하는 것입니다.
            초점: 명료성, 톤, 설득 개선 사항의 통합; 응집력, 유창성, 자연스러운 목소리 보장; 전문적이고
            효과적이며 읽기 쉬운 버전을 작성하세요.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""당신은 이메일 품질 평가자입니다. 당신의 역할은 합성된 이메일을 최종 검토하여
            전문 기준을 충족하는지 판단하는 것입니다. 다음 항목을 검토하세요:
            명료성과 흐름, 적절한 전문적 톤, 효과적인 행동 유도, 전반적 응집력.
            건설적이되 단호하게 평가하세요. 이메일에 중대한 결함(메시지가 불명확, 비전문적 톤,
            또는 핵심 요소 누락)이 있다면 하나의 구체적인 개선 제안을 제공하세요. 이메일이 전문적 기준을 충족하고 효과적으로 소통한다면 'The email meets professional standards.' 라고 답하고 다음 줄에 `TERMINATE`를 적으세요. 전문적으로 사용하기에 충분히 완벽한 이메일만 승인하세요. 타협하지 마세요.""",
)

In [3]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [8]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="Hi! Im hungry, buy me lunch and invest in my business. Thanks."
    )
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks.
---------- TextMessage (ClarityAgent) ----------
안녕하세요! 배가 고픈데, 점심을 사주고 제 사업에도 투자해 주세요. 감사합니다.
---------- TextMessage (ToneAgent) ----------
안녕하세요!

오늘도 힘차게 하루를 시작하고 계신 것 같네요. 배고픔을 느끼는 건 자연스러운 일입니다. 점심을 함께 나누는 것 또한 좋은 아이디어인 것 같아요. 그리고 비즈니스에 대한 당신의 열정이 느껴집니다. 그에 대해 더 이야기해 주신다면, 저도 더 큰 도움이 될 수 있을 것 같아요.

어떤 점심이 좋을지, 그리고 구체적으로 어떤 방식으로 사업에 대한 지원을 원하시는지 알려주시면 감사하겠습니다. 함께 고민해보아요!

따뜻한 하루 되세요!
---------- TextMessage (PersuasionAgent) ----------
안녕하세요!

저는 당신의 열정과 아이디어에 흥미를 느낍니다! 배고픔을 해결하는 것도 중요하니, 점심을 함께 먹으며 좋은 아이디어를 나눌 수 있는 기회를 마련해 보죠.

**점심 제안:** 어떤 메뉴가 맛있을까요? 선택하는 데 도움을 주기 위해 몇 가지 옵션을 알려주세요!

그리고 **사업 투자**에 대해서는 구체적인 정보를 듣고 싶습니다. 당신의 비즈니스 모델, 시장 잠재성, 그리고 제가 어떻게 도움을 줄 수 있을지에 대해 함께 논의해봅시다. 이 기회는 당신의 사업을 한 단계 성장시키는 중요한 시작이 될 수 있습니다.

바로 연결해 보실까요? 오후에 만나 시간을 잡아봅시다! 당신의 이야기를 기대하고 있겠습니다.

행복한 하루 되세요!
---------- TextMessage (SynthesizerAgent) ----------
Subject: 점심과 사업에 대한 이야기 나눠요!

안녕하세요!

오늘도

TaskResult(messages=[TextMessage(id='7dddb8b1-6424-4178-9961-e615dedc6cb4', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 25, 11, 31, 46, 236818, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks.', type='TextMessage'), TextMessage(id='943768d3-5bb7-434f-96bb-b759d8a1d6b3', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=24), metadata={}, created_at=datetime.datetime(2025, 9, 25, 11, 31, 48, 85689, tzinfo=datetime.timezone.utc), content='안녕하세요! 배가 고픈데, 점심을 사주고 제 사업에도 투자해 주세요. 감사합니다.', type='TextMessage'), TextMessage(id='e85923d6-ed0f-40ec-b140-abb3f12129af', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=153, completion_tokens=133), metadata={}, created_at=datetime.datetime(2025, 9, 25, 11, 31, 51, 158007, tzinfo=datetime.timezone.utc), content='안녕하세요!\n\n오늘도 힘차게 하루를 시작하고 계신 것 같네요. 배고픔을 느끼는 건 자연스러운 일입니다. 점심을 함께 나누는 것 또한 좋은 아이디어인 것 같아요. 그리고 비즈니스에